<a href="https://colab.research.google.com/github/OmdenaAI/omdena-colombia-socialnetwork/blob/task-1a-twitter-data-extraction/src/data/Scraped%20Notebooks/Interactive_Twitter_scrapers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes
* As a TODO, make use of the params that can be useful in the different scrappers


# Installations, imports and configurations

In [1]:
!git clone --depth=1 https://github.com/twintproject/twint.git
!cd /content/twint && pip3 install . -r requirements.txt -q
!pip install snscrape -q
!pip install stweet -q

Cloning into 'twint'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 1.3 MB 4.7 MB/s 
     |████████████████████████████████| 263 kB 47.9 MB/s 
     |████████████████████████████████| 364 kB 46.2 MB/s 
     |████████████████████████████████| 294 kB 72.9 MB/s 
     |████████████████████████████████| 142 kB 48.9 MB/s 
     |████████████████████████████████| 291 kB 59.9 MB/

In [3]:
import pandas as pd
from google.colab import files
import os

import twint
import nest_asyncio
nest_asyncio.apply()

import snscrape.modules.twitter as sntwitter

import stweet as st

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

CSV_PATH = "csv_files"
os.mkdir(CSV_PATH)

# Scraper-1 : TWINT 
One of the best API available and does not require any auth/ seperate script and overcomes most of twitter's official API's limitations

[Github](https://github.com/twintproject/twint)

[Analyzing Tweets with NLP in minutes with Spark, Optimus and Twint](https://towardsdatascience.com/analyzing-tweets-with-nlp-in-minutes-with-spark-optimus-and-twint-a0c96084995f)

In [ ]:
hashtag = "paronacional"  #@param {type: "string"}

config = twint.Config()
config.Search = hashtag
config.Pandas = True
config.Store_csv = True
config.Output = f"{CSV_PATH}/{hashtag}_tweets_twint.csv"

twint.run.Search(config)

## Preview

In [18]:
df = pd.read_csv(f"{CSV_PATH}/{hashtag}_tweets_twint.csv",index_col="id")
df.head()

,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1430373450099609600,1430373450099609600,2021-08-25 03:36:20 UTC,2021-08-25,03:36:20,0,194650157,saricastellanos,Sarita Gómez Castellanos,NaN,Construir una mejor Colombia no puede seguirno...,es,"[{'screen_name': 'angelloospinar', 'name': 'an...",[],[],0,2,2,"['paronacional', 'niperdonniolvido', 'dondeest...",[],https://twitter.com/saricastellanos/status/143...,False,NaN,1,https://pbs.twimg.com/ext_tw_video_thumb/14303...,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1430372059733954562,1430372059733954562,2021-08-25 03:30:48 UTC,2021-08-25,03:30:48,0,1386754505295937537,laucamkilla,Lau,NaN,Me uno para este jueves 26 de Agosto al Paro N...,es,[],[],[],0,20,45,['paronacional'],[],https://twitter.com/Laucamkilla/status/1430372...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1430367121997975553,1430367121997975553,2021-08-25 03:11:11 UTC,2021-08-25,03:11:11,0,175448120,fdovisual,fco,NaN,A LOS CHAR HAY QUE HECHAR!!! DE BARRANQUILL...,es,[],[],[],1,0,2,"['viva', 'paronacional']",[],https://twitter.com/fdovisual/status/143036712...,False,https://twitter.com/Yoya601/status/14303658602...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1430365585876082691,1430358490816602115,2021-08-25 03:05:05 UTC,2021-08-25,03:05:05,0,342728137,cu1cu2ta,Jesus Alberto Ramirez,NaN,"@sergio_fajardo Tan RESPONSABLE como usted, qu...",es,"[{'screen_name': 'rioslibresant', 'name': 'afe...",[],[],0,0,0,"['paronacional', 'duqueasesino', 'duquealacpi']",[],https://twitter.com/cu1cu2ta/status/1430365585...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'sergio_fajardo', 'name': 'Se...",NaN,NaN,NaN,NaN
1430364822231789571,1430364822231789571,2021-08-25 03:02:03 UTC,2021-08-25,03:02:03,0,342728137,cu1cu2ta,Jesus Alberto Ramirez,NaN,"Tan RESPONSABLE como usted, que le hizo el fav...",es,"[{'screen_name': 'rioslibresant', 'name': 'afe...",['https://twitter.com/sergio_fajardo/status/14...,[],0,1,1,"['paronacional', 'duqueasesino', 'duquealacpi']",[],https://twitter.com/cu1cu2ta/status/1430364822...,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [19]:
df.dtypes

1428854495258333185 2021-08-20 23:00:33 +0000 <neryrol> A quien se puede acudir por justicia si la primera en bloquearle el acceso a ella es la @PNCdeGuatemala   #RenunciaGiammattei #RenunciaConsuelo #DoneEstaElDinero #DondeEstanLasVacunas #ParoNacional
1428847761550675968 2021-08-20 22:33:47 +0000 <neryrol> Ya es hora de vacunar a todos...   nos estan matando sin vacunas.  @MinSaludGuate bola de ineptos.  #RenunciaGiammattei #RenunciaConsuelo #DoneEstaElDinero #DondeEstanLasVacunas #ParoNacional
1428834294760366083 2021-08-20 21:40:16 +0000 <enyapraez> Que aparezcan los 70 mil millones o #ParoNacional
1428831705452978190 2021-08-20 21:29:59 +0000 <ArmandoVillotaE> Ernesto "jugadita" Macias retira su candidatura a la presidencia de #Colombia. El aduras penas bachiller, nos privó de sus mafufes en la Casa de Nariño.  #ParoNacional
1428828500023078913 2021-08-20 21:17:15 +0000 <DianaRios3> No jodas!!!!  Ya no más !!! Al comité del paro No le basto con aumento de contagios por covid, la a

conversation_id      int64
created_at          object
date                object
time                object
timezone             int64
user_id              int64
username            object
name                object
place              float64
tweet               object
language            object
mentions            object
urls                object
photos              object
replies_count        int64
retweets_count       int64
likes_count          int64
hashtags            object
cashtags            object
link                object
retweet               bool
quote_url           object
video                int64
thumbnail           object
near               float64
geo                float64
source             float64
user_rt_id         float64
user_rt            float64
retweet_id         float64
reply_to            object
retweet_date       float64
translate          float64
trans_src          float64
trans_dest         float64
dtype: object

## Download

In [ ]:
files.download(f"{CSV_PATH}/{hashtag}_tweets_twint.csv")

# Scraper-2 SNS Scraper- 
This one is very useful as it can be used to scrape data from a variety of social media like Facebook, Instagram,Twitter,Reddit etc. Does not provide the location from where it was tweeted even though there are parameters to indicate . THe usage is quite easy to understand and implement. Has a limit of 100k tweets that can scraped within a day
## NOTE: If the limit is crossed warning will not displayed immediately and will take a lot of time before it gives an error so be aware of the limit whilst scraping . Also there is an issue whilst saving this as a csv file wherein special charcters like emojis will not be displayed 
[Github Repo](https://github.com/JustAnotherArchivist/snscrape) -  Have a look at twitter.py file to see what parameters can be scraped . I have used only a few of them here

In [ ]:
start_date = "2021-04-28" #@param {type:"date"}
end_date = "2021-08-24" #@param {type:"date"}
hashtag = "paronacional" #@param {type:"string"}
limit = 10 #@param{type:"integer"}
# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f"#{hashtag} since:{start_date} until:{end_date}").get_items()):
    if i>limit:
        break
    print(tweet,"#$")
    print(tweet.date,id,tweet.content,tweet.__str__)
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.username])
    
# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

## Preview

In [32]:
tweets_df2

,Datetime,Tweet Id,Text,Username
0,2021-08-23 23:38:04+00:00,1429951100371755011,Esos verdes no saben nada de Verde o se la fumaran verde más bien...\nEncuentra el logo del girasol al lado de la cara de partido Verde \n#paronacional https://t.co/tT6w2TEm1f,fdovisual
1,2021-08-23 23:30:54+00:00,1429949299392122885,"Claudia López al siquiatra, quién le regala una cita para corroborar que no es apta para el cargo mitomanía\n#paronacional https://t.co/2B8IPM7acg",fdovisual
2,2021-08-23 23:24:34+00:00,1429947704361238529,"Atentos reatentos, por favor \n#paronacional https://t.co/yVB5l2Kw2i",fdovisual
3,2021-08-23 23:15:38+00:00,1429945457103540228,#ParoNacional\n¿Que tal este póster de Uribe? https://t.co/GsEXrZh8Om,Javier78000
4,2021-08-23 23:02:24+00:00,1429942126192824320,"Mi propósito y el de @MovilidadCali es cumplir con la reposición de TODOS los semáforos afectados durante el #ParoNacional.\n\nNos reunimos con los contratistas que se encargan de esta labor, para realizar seguimiento a sus deberes y el de las aseguradoras. \n\n#TeQueremosCali💙❤️💚 https://t.co/M5WPf8UX3v",WMVallejo
5,2021-08-23 23:00:01+00:00,1429941527103553538,"🚨 No te pierdas hoy, a las 9 p.m., el programa @zonafranca con nuestra cofundadora @DianaSalinasP. Se hablará sobre los hechos ocurridos el pasado #19M en el Éxito de Calipso, Cali, durante el #ParoNacional. \n\n¿Ya leíste #ApocalipsoNow?👇 https://t.co/TSZmGwgCD0",cuestion_p
6,2021-08-23 22:39:50+00:00,1429936447583309825,"#AEstaHora⏰ | junto a @bienes_cali, el secretario @WMVallejo y subsecretaria @mdmsolanilla, realizan seguimiento a las obras de reposición de los semáforos por parte de las empresas operadoras, vandalizados durante el #ParoNacional. \n\n#TeQueremosCali💙❤️💚\n@JorgeIvanOspina https://t.co/VLnAlYxuee",MovilidadCali
7,2021-08-23 22:35:48+00:00,1429935430653386753,👇🏻👇🏻👇🏻👇🏻👏🏻👏🏻👏🏻✊✊✊✊✊🇨🇴🇨🇴🇨🇴🇨🇴🇨🇴🇨🇴 \n#ParoNacional https://t.co/GtioVPkcta,bluejoin19
8,2021-08-23 22:30:17+00:00,1429934043756437512,#DondeEstan | Se cumplen casi 4 meses de #ParoNacional y por lo menos 72 personas siguen desaparecidas ¿Por qué insisten en usar eufemismos para suavizar y negar la verdad? \n\nEl abogado Andrés Yasnó analiza el negacionismo del gobierno Duque ▶️https://t.co/z0sODFmf1F https://t.co/jGuijV3LSd,RazonPublica
9,2021-08-23 22:13:52+00:00,1429929909946884105,"#Colombia y la realidad sobre la #PrimeraLineaTerrorista, su financiamiento y las #FARC #ParoNacional = #Revolución Molecular Disipada https://t.co/oX1nTLrrBz",margeU87


In [33]:
tweets_df2.dtypes

Datetime    datetime64[ns, UTC]
Tweet Id                  int64
Text                     object
Username                 object
dtype: object

## Download

In [29]:
tweets_df2.to_csv(f"{CSV_PATH}/{hashtag}_tweets_snscrape.csv")
files.download(f"{CSV_PATH}/{hashtag}_tweets_snscrape.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Scraper-3 STweet
This is one of the best API's in terms of adapting to changes in Twitter's API ( a recent change affected a lot of popular API's to extract tweets) It can find the changes and report failures to find the point of failure so that the owner or any other user can make a fix . It is also quite customizable and is very simple code to use and explore ( unlike TWINT where the name of exact parameters need to be remembered) 
Has a dedicated documentation and examples to go through
[Github Repo](https://github.com/markowanga/stweet)

In [39]:
#start_date = "2021-04-28" #@param {type:"date"}
#end_date = "2021-08-24" #@param {type:"date"}
hashtag = "paronacional" #@param {type:"string"}

search_tweets_task = st.SearchTweetsTask(
    all_words=f"#{hashtag}"
)
tweets_collector = st.CollectorTweetOutput()

st.TweetSearchRunner(
    search_tweets_task=search_tweets_task,
    tweet_outputs=[tweets_collector, st.CsvTweetOutput(f"{CSV_PATH}/{hashtag}_tweets_stweet.csv")]
).run()

SearchTweetsResult(downloaded_count=597)

## Preview

In [40]:
df3 = pd.read_csv(f"{CSV_PATH}/{hashtag}_tweets_stweet.csv")
df3.head()

,created_at,id_str,conversation_id_str,full_text,lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media
0,2021-08-25T03:36:20+00:00,1430373450099609600,1430373450099609600,Construir una mejor Colombia no puede seguirnos costando la vida ni la integridad de más jóvenes!\n#ParoNacional #NiPerdonNiOlvido #DondeEstanLosDesaparecidos \nCréditos: @AngelloospinaR https://t.co/u1oEf4f3pE,es,False,False,4,7,0,0,NaN,NaN,NaN,194650157,saricastellanos,Sarita Gómez Castellanos,False,NaN,NaN,"#ParoNacional , #NiPerdonNiOlvido , #DondeEstanLosDesaparecidos",AngelloospinaR,NaN,"[{""url"": ""https://pbs.twimg.com/ext_tw_video_thumb/1430373270537265152/pu/img/H6OFTyPL-CTRJtyh.jpg"", ""type"": ""photo""}]"
1,2021-08-25T03:30:48+00:00,1430372059733954562,1430372059733954562,"Me uno para este jueves 26 de Agosto al Paro Nacional, no aguantamos más corrupción nos quieren quitar hasta lo que no tenemos. \n\n#ParoNacional",es,False,False,32,61,1,1,NaN,NaN,NaN,1386754505295937537,Laucamkilla,Lau,False,NaN,NaN,#ParoNacional,NaN,NaN,[]
2,2021-08-25T03:11:11+00:00,1430367121997975553,1430367121997975553,"A LOS CHAR HAY QUE HECHAR!!! DE BARRANQUILLA HAY QUE ECHAR,!!!\nHAY QUE HAY QUE HECHAR!!!... A LOS PERROS DE LOS CHAR!!!\n#VIVA EL PARO NACIONAL!!! \nBARRANQUILLA HAY QUE ACTUAR ...PUEBLO!!! nO MÁS VOTOS... \n#ParoNacional",es,False,False,0,2,1,0,1.430366e+18,NaN,NaN,175448120,fdovisual,fco,False,NaN,NaN,"#VIVA , #ParoNacional",NaN,NaN,[]
3,2021-08-25T03:05:05+00:00,1430365585876082691,1430358490816602115,"@sergio_fajardo Tan RESPONSABLE como usted, que le hizo el favor en las PASADAS ELECCIONES a NARCOPARAMILITARES en el poder. LOS CAMPESINOS despojados y ASESINADOS EN @RiosLibresAnt no lo olvidan, ni los jóvenes ASESINADOS #ParoNacional , esto son sus consecuencias #DuqueAsesino #DuqueALaCPI.",es,False,False,0,0,0,0,NaN,NaN,NaN,342728137,cu1cu2ta,Jesus Alberto Ramirez,False,1.430358e+18,25185308.0,"#ParoNacional , #DuqueAsesino , #DuqueALaCPI","sergio_fajardo , RiosLibresAnt",NaN,[]
4,2021-08-25T03:02:03+00:00,1430364822231789571,1430364822231789571,"Tan RESPONSABLE como usted, que le hizo el favor en las PASADAS ELECCIONES a NARCOPARAMILITARES en el poder. LOS CAMPESINOS despojados y ASESINADOS EN @RiosLibresAnt no lo olvidan, ni los jóvenes ASESINADOS #ParoNacional , esto son sus consecuencias #DuqueAsesino #DuqueALaCPI. https://t.co/Z7sFNkDnz1",es,False,False,1,1,0,0,NaN,NaN,NaN,342728137,cu1cu2ta,Jesus Alberto Ramirez,False,NaN,NaN,"#ParoNacional , #DuqueAsesino , #DuqueALaCPI",RiosLibresAnt,https://t.co/Z7sFNkDnz1,[]


## Download

In [ ]:
files.download(f"{CSV_PATH}/{hashtag}_tweets_stweet.csv")